In [3]:
! pip3 install -q langchain langchain-openai langchain-core langchain-community docx2txt pypdf langchain-chroma sentence_transformers python-dotenv

import langchain
print(langchain.__version__)

1.2.7


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.environ["LANGSMITH_ENDPOINT"])
print(os.environ["LANGCHAIN_TRACING_V2"])

https://api.smith.langchain.com
true


### What is Retrieval Augmented Generation (RAG)?

RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

### Overview of Langchain

Langchain is a framework for developing applications powered by language models. It provides a set of tools and abstractions that make it easier to build complex AI applications. Key features include:

- Modular components for common LLM tasks
- Built-in support for various LLM providers
- Tools for document loading, text splitting, and vector storage
- Abstractions for building conversational agents and question-answering systems

Call LLM

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm_response = llm.invoke("Tell me a joke about programming.")
print(f"1st llm_response: {llm_response}")
print(f"2nd llm_response: {llm_response.content}")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1st llm_response: content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D4tuCTUnmaStWItXhGfci2SJ6FUwD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1fd5-128e-7de1-861e-a4b552e37283-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
2nd llm_response: Why do programmers prefer dark mode?

Because light attract

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [7]:
chain = llm | output_parser
chain.invoke("Tell me a joke about programming.")

'Why do programmers prefer dark mode? Because the light attracts bugs!'

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
  phone_model: str = Field(description="The name and model of the mobile phone being reviewed")
  rating: int = Field(description="The rating of the mobile review from 1 to 5")
  pros: List[str] = Field(description="A list of positive things mentioned in the mobile review")
  cons: List[str] = Field(description="A list of negative things mentioned in the mobile review")
  summary: str = Field(description="Brief summary of the mobile review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous, colors pop like crazy.
Camera's insane too, especially at night - my Insta game's never been stronger. Battery life is solid, easily lasts me all day with heavy use.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from being perfect.
If you're due for an upgrade, definitely worth checkng out.
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)

output

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:2073: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


MobileReview(phone_model='Galaxy S21', rating=4, pros=['Gorgeous screen with vibrant colors', 'Impressive camera performance, especially at night', 'Solid battery life'], cons=['Pricey', 'No charger included', 'New button layout takes getting used to'], summary='The Galaxy S21 is a great phone with a few quirks. It has a gorgeous screen, impressive camera, and solid battery life. However, the high price, lack of included charger, and new button layout may be drawbacks for some users. Overall, a solid 4 out of 5 rating - worth checking out for those due for an upgrade.')

## Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}.")

# prompt = ChatPromptTemplate.from_template([
#     ("system", "You are a helpful assistant that extracts structured data from mobile phone reviews."),
#     ("user", "Extract the following review into structured data:\n{review_text}")
# ])
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about programming.', additional_kwargs={}, response_metadata={})])

In [11]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "car drivers"})

'Why did the car driver bring a map to the bar? \nBecause he heard the drinks were on the house!'

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# Initilize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "artificial intelligence"})
print(result)


Why did the robot go on a diet? 

Because it had too many bytes!


## LLM Messages

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me a joke about robots.")
llm.invoke([system_message, human_message])

AIMessage(content='Why was the robot so bad at soccer?\n\nBecause it kept kicking up sparks instead of the ball!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 27, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1590f93f9d', 'id': 'chatcmpl-D4z7G7Mvr4bJGeWJUKvQJGzg0zDxj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2106-b33b-7420-9f12-78439452bc24-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 27, 'output_tokens': 20, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})